In [9]:
import glob
import tqdm
import cv2
import json
import os

### common part
---

In [ ]:
### for read of raw image width and length
# import json
# with open('./tile_round1_train_20201231/train_annos.json', 'r') as fp:
#      img_info_dic=json.load(fp)

# img_info_dic={d['name'].split('.j')[0]:d for d in img_info_dic}

In [16]:
# parameters
path='../code/yolov5/runs/detect/exp18/labels/'
img_path='./tcdata/tile_round1_testA_20201231/testA_imgs/'
img_path='./tcdata/tile_round1_testB_20210128/testB_imgs/'

output_path='./round1_testB_labels_63/'
output_path='./round1_testB_labels_59/'
output_path='./round1_testB_labels_62/'

if not os.path.exists(output_path): 
    os.makedirs(output_path)

target_size=1280

In [17]:
sep_labels=glob.glob(path+'*.txt')

In [18]:
len(sep_labels)

17517

In [13]:
sep_labels[1]

'./yolov5/runs/detect/exp17/labels\\258_142_t20201202094108644_CAM3_4096_3500_0_2560.txt'

In [62]:
### old version gen yolo anno
# for label_path in sep_labels:
#     img_name=label_path.split('\\')[1].split('.t')[0]
#     img_name=img_name.split('_')
#     x0,y0=img_name[-2:]
#     img_name='_'.join(img_name[:-4])

#     with open(label_path,'r') as f:
#         lines=f.readlines()

#     for line in lines:
#         l,x,y,w,h=line.strip().split(' ')
#         img_info=img_info_dic[img_name]
#         x=(float(x)*target_size+float(x0))/img_info['image_width']
#         y=(float(y)*target_size+float(y0))/img_info['image_height']
#         w=(float(w)*target_size)/img_info['image_width']
#         h=(float(h)*target_size)/img_info['image_height']

#         out_string="{} {:.6f} {:.6f} {:.6f} {:.6f}\n".format(l,x,y,w,h)
#         with open(output_path+img_name+'.txt','a+') as f:
#             f.write(out_string)        

### without mask (finally chosen)
---

In [19]:
### gen json without mask
results=[]
threshold=0.01

for label_path in tqdm.tqdm(sep_labels):
#     label_path=sep_labels[1]
    img_name=label_path.split('\\')[1].split('.t')[0]
    img_name=img_name.split('_')
    x0,y0=img_name[-2:]
    img_w=img_name[-4]
    img_h=img_name[-3]
    img_name='_'.join(img_name[:-4])
    
    with open(label_path,'r') as f:
        lines=f.readlines()

    for line in lines:
        l,x,y,w,h,score=line.strip().split(' ')
        score=float(score)
        if score>=threshold:
            x=round(float(x)*target_size+float(x0))
            y=round(float(y)*target_size+float(y0))
            w=round(float(w)*target_size)
            h=round(float(h)*target_size)
            temp_dict={'name': img_name+'.jpg',\
#                        'image_height': int(img_h),\
#                        'image_width': int(img_w),\
                       'category': int(l),\
                       'bbox': [x-w//2, y-h//2, x+w//2, y+h//2],\
                       'score':score }
            results.append(temp_dict)
        
json_str = json.dumps(results)
with open('../prediction_result/result.json','w') as f:
    f.write(json_str)

100%|██████████████████████████████████████████████████████████████████████████| 17517/17517 [00:02<00:00, 7473.11it/s]


In [14]:
len(results)

3432

### with mask
---

In [9]:
### gen json with mask
results=[]
threshold=0.01

def get_mask(img,halfsize=100):
    #获取mask
    height, width = img.shape[:2]
    lower_color=img[:halfsize,:,:].min(axis=1).min(axis=0)
    upper_color=img[:halfsize,:,:].max(axis=1).max(axis=0)
    # lower_color=img[rows//2-halfsize:rows//2+halfsize,cols//2-halfsize:cols//2+halfsize,:].min(axis=1).min(axis=0)
    # upper_color=img[rows//2-halfsize:rows//2+halfsize,cols//2-halfsize:cols//2+halfsize,:].max(axis=1).max(axis=0)
    mask = cv2.inRange(img, lower_color, upper_color)
    return mask




for label_path in tqdm.tqdm(sep_labels):
#     label_path=sep_labels[1]
    img_name=label_path.split('\\')[1].split('.t')[0]
    img_name=img_name.split('_')
    x0,y0=img_name[-2:]
    img_w=img_name[-4]
    img_h=img_name[-3]
    img_name='_'.join(img_name[:-4])
    
    img=cv2.imread(img_path+img_name+'.jpg')
    mask=get_mask(img)

    with open(label_path,'r') as f:
        lines=f.readlines()

    for line in lines:
        l,x,y,w,h,score=line.strip().split(' ')
        score=float(score)
        if score>=threshold:
            x=round(float(x)*target_size+float(x0))
            y=round(float(y)*target_size+float(y0))
            w=round(float(w)*target_size)
            h=round(float(h)*target_size)
            if mask[y:y+h,x:x+w].mean(axis=1).min()==0:
                temp_dict={'name': img_name+'.jpg',\
#                            'image_height': int(img_h),\
#                            'image_width': int(img_w),\
                           'category': int(l),\
                           'bbox': [x-w//2, y-h//2, x+w//2, y+h//2],\
                           'score':score }
                results.append(temp_dict)
        
json_str = json.dumps(results)
with open('./results_w_mask_20210125.json','w') as f:
    f.write(json_str)

100%|██████████████████████████████████████████████████████████████████████████| 23816/23816 [2:48:27<00:00,  2.36it/s]


In [11]:
len(results)

1570

In [20]:
### gen yolo anno
for label_path in tqdm.tqdm(sep_labels):
    img_name=label_path.split('\\')[1].split('.t')[0]
    img_name=img_name.split('_')
    x0,y0=img_name[-2:]
    img_w=float(img_name[-4])
    img_h=float(img_name[-3])
    img_name='_'.join(img_name[:-4])

    with open(label_path,'r') as f:
        lines=f.readlines()

    for line in lines:
        l,x,y,w,h,score=line.strip().split(' ')
        x=(float(x)*target_size+float(x0))/img_w
        y=(float(y)*target_size+float(y0))/img_h
        w=(float(w)*target_size)/img_w
        h=(float(h)*target_size)/img_h

        out_string="{} {:.6f} {:.6f} {:.6f} {:.6f}\n".format(l,x,y,w,h)
        with open(output_path+img_name+'.txt','a+') as f:
            f.write(out_string)        

100%|██████████████████████████████████████████████████████████████████████████| 17517/17517 [00:06<00:00, 2510.31it/s]
